<a href="https://colab.research.google.com/github/411021210/AIcupTutorial/blob/main/%E3%80%8CDan_Training_Tutorial_checkpoint_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
tf.__version__

'2.14.0'

In [8]:
!pip install google.colab #如未安裝取消註解後執行
import os

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/AICup')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`.

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [9]:
df = open('./CSVs/Tutorial_dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [10]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [11]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [12]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 1280, Total Moves: 297110


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [13]:
x = []
y = []
for game in games[:500]:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

In [14]:
x.shape

(118278, 19, 19, 4)

In [ ]:
y.shape

(118777,)

In [ ]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [ ]:
def create_model():
    inputs = Input(shape=(19, 19, 4))
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 32)        6304      
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 32)        50208     
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 32)        25632     
                                                                 
 conv2d_3 (Conv2D)           (None, 19, 19, 32)        25632     
                                                                 
 conv2d_4 (Conv2D)           (None, 19, 19, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 19, 19, 1)         289   

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 128,
    epochs = 10,
    validation_data=(x_val, y_val),
)

Epoch 1/10
836/836 [==============================] - 15s 14ms/step - loss: 4.8747 - accuracy: 0.0469 - val_loss: 4.5146 - val_accuracy: 0.0753
Epoch 2/10
836/836 [==============================] - 11s 14ms/step - loss: 4.4061 - accuracy: 0.0832 - val_loss: 4.4040 - val_accuracy: 0.0860
Epoch 3/10
836/836 [==============================] - 11s 14ms/step - loss: 4.2904 - accuracy: 0.1002 - val_loss: 4.3429 - val_accuracy: 0.0974
Epoch 4/10
836/836 [==============================] - 11s 14ms/step - loss: 4.1692 - accuracy: 0.1231 - val_loss: 4.2078 - val_accuracy: 0.1219
Epoch 5/10
836/836 [==============================] - 11s 14ms/step - loss: 4.0474 - accuracy: 0.1439 - val_loss: 4.1036 - val_accuracy: 0.1414
Epoch 6/10
836/836 [==============================] - 11s 14ms/step - loss: 3.9493 - accuracy: 0.1646 - val_loss: 4.0474 - val_accuracy: 0.1534
Epoch 7/10
836/836 [==============================] - 11s 14ms/step - loss: 3.8689 - accuracy: 0.1814 - val_loss: 4.0065 - val_accuracy:

In [ ]:
model.save('./model_dan_tutorial.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!